In [ ]:
pip install opensearch-py openai sentence-transformers --index-url https://pypi.org/simple

In [ ]:
from opensearchpy import OpenSearch, helpers
import os
from sentence_transformers import SentenceTransformer
import json

model = SentenceTransformer("all-MiniLM-L6-v2")


host = 'localhost'
port = 9200
auth = ('admin', 'admin') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [ ]:
query = {
  "query": {
    "match": {
      "title": "quick fox"
    }
  }
}

response = client.search(
    body = query,
    index = 'my_index',
    params = {'explain': 'true', 'pretty': 'true'}
)

print(response)


In [ ]:
index_name = "oci-opensearch-runbooks"
index_body = {
  "settings": {
    "index": {
      "number_of_shards": 4,
      "knn": True
    }
  },
  "mappings": {
    "properties": {
      "runbook_text": {
          "type": "text"
      },
      "runbook_name": {
          "type": "text"
      },
      "runbook_vector": {
        "type": "knn_vector",
        "dimension": 384,
        "method": {
          "space_type": "l2",
          "name": "hnsw"
        }     
      }
    }
  }
}

client.indices.create(
    index = index_name,
    body = index_body
)

In [ ]:
base_dir = "/Users/sannaren/mp_repos/runbooks/search-howtos/"
runbooks = [f for f in os.listdir(base_dir) if f.endswith(".md")]
print(runbooks)

documents = []
for r in runbooks:
    r_path = base_dir + "/" + r
    f = open(r_path)
    r_text = f.read()
    f.close()
    r_vector = model.encode(r_text)
    document = {"_index": index_name, "_source": {"runbook_name": r_path, "runbook_text": r_text, "runbook_vector": r_vector}}
    documents.append(document)

response = helpers.bulk(client, documents)
print(response)


In [ ]:
query = ''''How to fetch customer details of a cluster'''
k = 2
vector = model.encode(query)
search_body = {
  "size": 2,
  "query": {
    "knn": {
      "runbook_vector": {
        "vector": vector,
        "k": k
      }
    }
  }
}

response = client.search(
    body=search_body,
    index=index_name
)

json_string = json.dumps(response, indent=2)

print(json_string)